In [1]:
import sys
# Importar tu modelo corregido (asegúrate de que el archivo .py esté en la carpeta)
sys.path.append("KAN_models")
from SBTAYLOR_KAN import Net

In [2]:
from torchview import draw_graph
import torch

model = Net() # Tu modelo SBTAYLOR-KAN
batch_size = 2
# Genera el diagrama
model_graph = draw_graph(model, input_size=(batch_size, 3, 224, 224), expand_nested=True)
model_graph.visual_graph.render("arquitectura_kan", format="png")

Nueva dimensión para FC1 (con GAP): 128


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [4]:
!pip install onnx onnxscript

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ------ --------------------------------- 2.6/16.5 MB 25.4 MB/s eta 0:00:01
   ------------ --------------------------- 5.2/16.5 MB 13.9 MB/s eta 0:00:01
   --------------- ------------------------ 6.3/16.5 MB 10.7 MB/s eta 0:00:01
   -------------------- ------------------- 8.7/16.5 MB 10.5 MB/s eta 0:00:01
   ------------------------ --------------- 10.0/16.5 MB 9.7 MB/s eta 0:00:01
   --------------------------- ------------ 11.3/16.5 MB 9.0 MB/s eta 0:00:01
   ------------------------------- -------- 12.8/16.5 MB 8.6 MB/s eta 0:00:01
   ---------------------------------- ----- 14.2/16.5 MB 8.3 MB/s eta 0:00:01
   -------------------------------------- - 15.7/16.5 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 16.5/16.5 MB 7.9 MB/s  0:00:02
   ---------------------------------------- 0.0/693.4 kB ? eta -:--:--
   ---------------------------------------- 693.4/693.4 kB 6.8 MB/s  0:00:00

  At

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.2 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf<4.22,>=4.21.6; python_version < "3.11", but you have protobuf 6.33.2 which is incompatible.


In [5]:
import torch

# 1. Asegúrate de que el modelo esté en modo evaluación
model.eval()

# 2. Crea un tensor "dummy" con la forma de entrada (Batch, Canales, H, W)
dummy_input = torch.randn(1, 3, 224, 224).to(next(model.parameters()).device)

# 3. Exportar
onnx_path = "mi_modelo_kan.onnx"
torch.onnx.export(
    model, 
    dummy_input, 
    onnx_path, 
    verbose=False, 
    input_names=['input_imagen'], 
    output_names=['output_clases'],
    opset_version=12 # Versión estándar de compatibilidad
)

print(f"✅ Modelo exportado con éxito en: {onnx_path}")

W0105 03:09:34.563000 45556 Lib\site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 12).
Failed to convert the model to the target version 12 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "c:\Users\Admin-Cidis\Documents\mai\cotton-leaf-defect-classification\.venv\lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in 

Applied 8 of general pattern rewrite rules.
✅ Modelo exportado con éxito en: mi_modelo_kan.onnx


In [6]:
from torchsummary import summary

# Asegúrate de que el modelo esté en el dispositivo correcto (cuda o cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Genera el resumen
summary(model, input_size=(3, 224, 224))

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       896
|    └─BatchNorm2d: 2-2                  64
|    └─ReLU: 2-3                         --
|    └─MaxPool2d: 2-4                    --
|    └─Conv2d: 2-5                       18,496
|    └─BatchNorm2d: 2-6                  128
|    └─ReLU: 2-7                         --
|    └─MaxPool2d: 2-8                    --
|    └─Conv2d: 2-9                       73,856
|    └─BatchNorm2d: 2-10                 256
|    └─ReLU: 2-11                        --
|    └─MaxPool2d: 2-12                   --
|    └─Conv2d: 2-13                      147,584
|    └─BatchNorm2d: 2-14                 256
|    └─ReLU: 2-15                        --
|    └─MaxPool2d: 2-16                   --
├─TaylorSeriesApproximation: 1-2         --
├─AdaptiveAvgPool2d: 1-3                 --
├─KANLinear: 1-4                         --
|    └─SiLU: 2-17                        --
├─Dropout:

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       896
|    └─BatchNorm2d: 2-2                  64
|    └─ReLU: 2-3                         --
|    └─MaxPool2d: 2-4                    --
|    └─Conv2d: 2-5                       18,496
|    └─BatchNorm2d: 2-6                  128
|    └─ReLU: 2-7                         --
|    └─MaxPool2d: 2-8                    --
|    └─Conv2d: 2-9                       73,856
|    └─BatchNorm2d: 2-10                 256
|    └─ReLU: 2-11                        --
|    └─MaxPool2d: 2-12                   --
|    └─Conv2d: 2-13                      147,584
|    └─BatchNorm2d: 2-14                 256
|    └─ReLU: 2-15                        --
|    └─MaxPool2d: 2-16                   --
├─TaylorSeriesApproximation: 1-2         --
├─AdaptiveAvgPool2d: 1-3                 --
├─KANLinear: 1-4                         --
|    └─SiLU: 2-17                        --
├─Dropout: